In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/My_config/vfnet_x101_64x4d_fpn_ver2.py')

PREFIX = '../../input/data/'

epoch = 21

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.val.classes = classes
cfg.data.test.classes = classes


#cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

#cfg.model.roi_head.bbox_head.num_classes = 11

#cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
for v in data_loader:
    print(v.keys())
    print(len(v['img_metas']))
    print(v['img_metas'][0])
    break

dict_keys(['img_metas', 'img'])
8
DataContainer([[{'filename': '/opt/ml/pstage3/input/data/batch_01_vt/0021.jpg', 'ori_filename': 'batch_01_vt/0021.jpg', 'ori_shape': (512, 512, 3), 'img_shape': (307, 307, 3), 'pad_shape': (320, 320, 3), 'scale_factor': array([0.5996094, 0.5996094, 0.5996094, 0.5996094], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([117.302, 112.092, 106.659], dtype=float32), 'std': array([53.759, 52.954, 55.223], dtype=float32), 'to_rgb': True}}]])


In [4]:
checkpoint_path

'./work_dirs/vfnet_x101_64x4d_fpn_ver2/epoch_21.pth'

In [5]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader


In [6]:
output = single_gpu_test(model, data_loader, show_score_thr=0.3)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 0.7 task/s, elapsed: 1264s, ETA:     0s

In [7]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.11261497 0.0 336.2115 86.24252 450.92987 0...,batch_01_vt/0021.jpg
1,0 0.1892994 285.63828 339.294 306.77774 376.12...,batch_01_vt/0028.jpg
2,0 0.16174813 235.3119 461.15344 303.19208 511....,batch_01_vt/0031.jpg
3,0 0.09282366 294.43723 156.67241 363.39825 234...,batch_01_vt/0032.jpg
4,1 0.75377715 388.095 271.9264 470.4365 345.986...,batch_01_vt/0070.jpg
